In [38]:
import pandas as pd
import sqlalchemy as sql
import pyads
import pymssql
import pypyodbc as odbc

In [39]:
#plc = pyads.Connection('5.119.159.52.1.1', pyads.PORT_TC3PLC1)

#plc.open()

In [40]:
#plc.read_by_name('GVL.EL3443.MDP5001_341_Input.MDP5001_341_L1_Basic_Voltage')

In [41]:
#plc.close()

In [42]:
Driver = 'SQL SERVER'
Server = 'AYO-NB8'
Database = 'PyADS'

connection_string = f"""
    DRIVER={{{Driver}}};
    SERVER={Server};
    Database={Database};
    Trust_Connection=yes
"""
conn = odbc.connect(connection_string)

In [43]:
#query = "INSERT INTO EL3443 (id, fVoltage, fCurrent, fAcPower, fApPower, fRePower, fPWF, fFreq) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
#val = (4, 1, 1, 1, 1, 1, 1, 1)

#cursor.execute(query, val)
#conn.commit()

In [44]:
cursor = conn.cursor()
new_query = "SELECT * FROM EL3443"
cursor.execute(new_query)
#result = cursor.fetchall()
data = pd.DataFrame(cursor, columns=[i[0] for i in cursor.description])
data = data.set_index('id')
data.head()

,fvoltage,fcurrent,facpower,fappower,frepower,fpqf,ffreq
id,,,,,,,
1,242.704193,-0.008795,-0.027324,2.134490,2.134315,0.996691,50.031918
2,242.587845,-0.008803,-0.051194,2.135606,2.134993,0.996662,50.030983
3,242.713181,-0.008814,-0.045052,2.139300,2.138825,0.996662,50.028816
4,242.699081,-0.008842,-0.049646,2.145965,2.145391,0.996659,50.027889
5,242.760391,-0.008795,-0.064997,2.134982,2.133992,0.996655,50.025177


In [45]:
conn.close()